# Basics: how NMEA Parser work, with example.

All NMEA Strings are made out of several parts:
- A '$' sign
- A Device (aka talker) ID
- A Sentence ID
- The Sentence data
- A Checksum, preceded with a star
- A String terminator, usually CR-LF (Carriage Return, and Line Feed)

Example, RMC String:
```
 $GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W,T*6A
 || |   |                                                            |
 || |   |                                                            Checksum, 6A
 || |   Data, comma-separated, 123519,...,T
 || Sentence ID, RMC
 |Talker ID, GP
 Dollar sign
```


The Data, per the NMEA Spec:

```
                                                                  12
       1      2 3        4 5         6 7     8     9      10    11
$GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W,T*6A
       |      | |        | |         | |     |     |      |     | |
       |      | |        | |         | |     |     |      |     | Type
       |      | |        | |         | |     |     |      |     Variation sign
       |      | |        | |         | |     |     |      Variation value
       |      | |        | |         | |     |     Date DDMMYY
       |      | |        | |         | |     COG
       |      | |        | |         | SOG
       |      | |        | |         Longitude Sign
       |      | |        | Longitude Value
       |      | |        Latitude Sign
       |      | Latitude value
       |      Active or Void
       UTC

```
The Sentence IDs, and their corresponding content and structure are defined by the NMEA specification.

## The checksum

The first thing to do is to validate the checksum; this will tell you if the sentence is valid, and is worth parsing.

The checksum validation concerns the part of the string _after_ the `$` sign, and _before_ the `*`.  
In the above, this would be `GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W,T`.

The checksum is the HexaDecimal value of the XOR (aka eXclusive OR) value of each byte of the string to validate.

Here is a Java example of such an operation:

In [1]:
String nmeaSentence = "$GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W*6A\r\n"; // No Type here
String str = nmeaSentence.substring(1, nmeaSentence.indexOf("*")); // Assume there IS a star
System.out.printf("Validating %s\n", str);

int cs = 0;
char[] ca = str.toCharArray();
cs = ca[0];
for (int i = 1; i < ca.length; i++) {
  cs = cs ^ ca[i]; // XOR
  System.out.printf("...Checksum is now 0x%02X \n", cs);
}
System.out.printf("Final Checksum 0x%02X (decimal %d)\n", cs, cs);

Validating GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W
...Checksum is now 0x17 
...Checksum is now 0x45 
...Checksum is now 0x08 
...Checksum is now 0x4B 
...Checksum is now 0x67 
...Checksum is now 0x56 
...Checksum is now 0x64 
...Checksum is now 0x57 
...Checksum is now 0x62 
...Checksum is now 0x53 
...Checksum is now 0x6A 
...Checksum is now 0x46 
...Checksum is now 0x07 
...Checksum is now 0x2B 
...Checksum is now 0x1F 
...Checksum is now 0x27 
...Checksum is now 0x17 
...Checksum is now 0x20 
...Checksum is now 0x0E 
...Checksum is now 0x3E 
...Checksum is now 0x0D 
...Checksum is now 0x35 
...Checksum is now 0x19 
...Checksum is now 0x57 
...Checksum is now 0x7B 
...Checksum is now 0x4B 
...Checksum is now 0x7A 
...Checksum is now 0x4B 
...Checksum is now 0x78 
...Checksum is now 0x49 
...Checksum is now 0x67 
...Checksum is now 0x57 
...Checksum is now 0x67 
...Checksum is now 0x57 
...Checksum is now 0x7B 
...Checksum is now 0x3E 
...Checksum is now 0x12 

java.io.PrintStream@395716dc

If the calculated checksum is the same as the one provided in the string itself, then the string is valid. We are looking for a `6A` here.

The string happens here to be valid, we can proceed to parsing.

Let's split the data into an array of elements:

In [2]:
String[] data = str.split(",");

In [3]:
Arrays.asList(data).forEach(System.out::println);

GPRMC
123519
A
4807.038
N
01131.000
E
022.4
084.4
230394
003.1
W


Then, the NMEA spec tells us what element corresponds to what data (starting with index 1).  Look [here](https://gpsd.gitlab.io/gpsd/NMEA.html#_rmc_recommended_minimum_navigation_information) for RMC structure.

Let's try to parse a `RMC` sentence.  
We can give a name to each element of the data array.

In [4]:
final int RMC_UTC = 1;
final int RMC_ACTIVE_VOID = 2;
final int RMC_LATITUDE_VALUE = 3;
final int RMC_LATITUDE_SIGN = 4;
final int RMC_LONGITUDE_VALUE = 5;
final int RMC_LONGITUDE_SIGN = 6;
final int RMC_SOG = 7;
final int RMC_COG = 8;
final int RMC_DDMMYY = 9;
final int RMC_VARIATION_VALUE = 10;
final int RMC_VARIATION_SIGN = 11;
final int RMC_TYPE = 12;

We create static methods to process the checksum, and others...

In [5]:
static int calculateCheckSum(String str) {
    int cs = 0;
    char[] ca = str.toCharArray();
    cs = ca[0];
    for (int i = 1; i < ca.length; i++) {
        cs = cs ^ ca[i]; // XOR
    }
    return cs;
}

static boolean validCheckSum(String data) {
    String sentence = data.trim();
    boolean b = false;
    try {
        int starIndex = sentence.indexOf("*");
        if (starIndex < 0) {
            return false;
        }
        String csKey = sentence.substring(starIndex + 1);
        int csk = Integer.parseInt(csKey, 16);
        String str2validate = sentence.substring(1, sentence.indexOf("*"));
        int calcCheckSum = calculateCheckSum(str2validate);
        b = (calcCheckSum == csk);
    } catch (Exception ex) {
        System.err.println("Oops:" + ex.getMessage());
    }
    return b;
}

static double sexToDec(String degrees, String minutes) {
    double ret;
    try {
        double deg = Double.parseDouble(degrees);
        double min = Double.parseDouble(minutes);
        min *= (10.0 / 6.0);
        ret = deg + (min / 100D);
    } catch (NumberFormatException nfe) {
        nfe.printStackTrace();
        System.err.println("Degrees:" + degrees);
        System.err.println("Minutes:" + minutes);
        throw new RuntimeException("Bad number [" + degrees + "] [" + minutes + "]");
    }
    return ret;
}

Then we can define a class(es) to hold the parsed data

In [6]:
import java.text.SimpleDateFormat;

static class GeoPos {
    double lat;
    double lng;
    public GeoPos(double l, double g) {
        this.lat = l;
        this.lng = g;
    }
}

class RMC {
    GeoPos gp = null;
    double sog = -1D;
    double cog = -1D;

    boolean valid = false; // False means warning.

    Date rmcDate = null;
    Date rmcTime = null;
    double declination = -Double.MAX_VALUE;
    
    public enum RMC_TYPE {
        AUTONOMOUS,
        DIFFERENTIAL,
        ESTIMATED,
        NOT_VALID,
        SIMULATOR
    }

    RMC_TYPE rmcType = null;

    final static SimpleDateFormat SDF = new SimpleDateFormat("E dd-MMM-yyyy HH:mm:ss.SS");
    static {
        SDF.setTimeZone(TimeZone.getTimeZone("Etc/UTC"));
    }
}

And we can start parsing. We write a static method for this operation.

In [7]:
import java.text.NumberFormat;
import java.util.Calendar;
import java.util.Date;
import java.util.Locale;
import java.util.TimeZone;

final static NumberFormat NF = NumberFormat.getInstance(Locale.ENGLISH);

static RMC parseRMC(String str) {
    RMC rmc = null;
    if (str.length() < 6 || !str.contains("*")) {
        return null;
    }
    if (!validCheckSum(str)) {
        return null;
    }
    String s = str.substring(0, str.indexOf("*"));
    try {
        if (s.contains("RMC,")) {
            rmc = new RMC();

            String[] data = s.split(",");
            rmc.valid = (data[RMC_ACTIVE_VOID].equals("A")); // Active. Does not prevent the date and time from being available.
            if (data[RMC_UTC].length() > 0) { // Time and Date
                double utc = 0D;
                try {
                    utc = NF.parse(data[RMC_UTC]).doubleValue();
                } catch (Exception ex) {
                    System.out.println("data[1] in StringParsers.parseRMC");
                }
                int h = (int) (utc / 10_000);
                int m = (int) ((utc - (10_000 * h)) / 100);
                float sec = (float) (utc % 100f);

                // System.out.println("Data[1]:" + data[1] + ", h:" + h + ", m:" + m + ", s:" + sec);

                Calendar local = Calendar.getInstance(TimeZone.getTimeZone("Etc/UTC"));
                local.set(Calendar.HOUR_OF_DAY, h);
                local.set(Calendar.MINUTE, m);
                local.set(Calendar.SECOND, (int) Math.round(sec));
                local.set(Calendar.MILLISECOND, 0);
                if (data[RMC_DDMMYY].length() > 0) {
                    int d = 1;
                    try {
                        d = Integer.parseInt(data[RMC_DDMMYY].substring(0, 2));
                    } catch (Exception ex) {
                        ex.printStackTrace();
                    }
                    int mo = 0;
                    try {
                        mo = Integer.parseInt(data[RMC_DDMMYY].substring(2, 4)) - 1;
                    } catch (Exception ex) {
                        ex.printStackTrace();
                    }
                    int y = 0;
                    try {
                        y = Integer.parseInt(data[RMC_DDMMYY].substring(4));
                    } catch (Exception ex) {
                        ex.printStackTrace();
                    }
                    if (y > 50) {
                        y += 1900;
                    } else {
                        y += 2_000;
                    }
                    local.set(Calendar.DATE, d);
                    local.set(Calendar.MONTH, mo);
                    local.set(Calendar.YEAR, y);
                    Date rmcDate = local.getTime();
                    rmc.rmcDate = rmcDate;
                }
                Date rmcTime = local.getTime();
                rmc.rmcTime = rmcTime;
            }
            if (data[RMC_LATITUDE_VALUE].length() > 0 && data[RMC_LONGITUDE_VALUE].length() > 0) {
                String deg = data[RMC_LATITUDE_VALUE].substring(0, 2);
                String min = data[RMC_LATITUDE_VALUE].substring(2);
                double l = sexToDec(deg, min);
                if ("S".equals(data[RMC_LATITUDE_SIGN])) {
                    l = -l;
                }
                deg = data[RMC_LONGITUDE_VALUE].substring(0, 3);
                min = data[RMC_LONGITUDE_VALUE].substring(3);
                double g = sexToDec(deg, min);
                if ("W".equals(data[RMC_LONGITUDE_SIGN])) {
                    g = -g;
                }
                rmc.gp = new GeoPos(l, g);
            }
            if (data[RMC_SOG].length() > 0) {
                double speed = 0;
                try {
                    speed = NF.parse(data[RMC_SOG]).doubleValue();
                } catch (Exception ex) {
                    ex.printStackTrace();
                }
                rmc.sog = speed;
            }
            if (data[RMC_COG].length() > 0) {
                double cog = 0;
                try {
                    cog = NF.parse(data[RMC_COG]).doubleValue();
                } catch (Exception ex) {
                    ex.printStackTrace();
                }
                rmc.cog = cog;
            }
            if (data[RMC_VARIATION_VALUE].length() > 0 && data[RMC_VARIATION_SIGN].length() > 0) {
                double d = -Double.MAX_VALUE;
                try {
                    d = NF.parse(data[RMC_VARIATION_VALUE]).doubleValue();
                } catch (Exception ex) {
                    ex.printStackTrace();
                }
                if ("W".equals(data[RMC_VARIATION_SIGN])) {
                    d = -d;
                }
                rmc.declination = d;
            }
            if (data.length > 12) { // Can be missing
                switch (data[RMC_TYPE]) {
                    case "A":
                        rmc.rmcType = RMC.RMC_TYPE.AUTONOMOUS;
                        break;
                    case "D":
                        rmc.rmcType = RMC.RMC_TYPE.DIFFERENTIAL;
                        break;
                    case "E":
                        rmc.rmcType = RMC.RMC_TYPE.ESTIMATED;
                        break;
                    case "N":
                        rmc.rmcType = RMC.RMC_TYPE.NOT_VALID;
                        break;
                    case "S":
                        rmc.rmcType = RMC.RMC_TYPE.SIMULATOR;
                        break;
                    default:
                        rmc.rmcType = null;
                        break;
                }
            }
        }
    } catch (Exception e) {
        System.err.println("In parseRMC for " + str.trim() + ", " + e.toString());
        e.printStackTrace();
    }
    return rmc;
}

We're now good to start parsing

In [8]:
System.out.println("Parsing " + nmeaSentence);

RMC rmc = parseRMC(nmeaSentence);

// Display members
System.out.println("Valid:\t" + rmc.valid);
System.out.println("Lat:\t" + rmc.gp.lat);
System.out.println("Lng:\t" + rmc.gp.lng);
System.out.println("Speed over Ground:\t" + rmc.sog);
System.out.println("Course over Ground:\t" + rmc.cog);

System.out.println("UTC Date:\t" + RMC.SDF.format(rmc.rmcDate));
System.out.println("UTC Time:\t" + RMC.SDF.format(rmc.rmcTime));

System.out.println("Declination:\t" + rmc.declination);

Parsing $GPRMC,123519,A,4807.038,N,01131.000,E,022.4,084.4,230394,003.1,W*6A

Valid:	true
Lat:	48.1173
Lng:	11.516666666666667
Speed over Ground:	22.4
Course over Ground:	84.4
UTC Date:	Wed 23-Mar-1994 12:35:19.00
UTC Time:	Wed 23-Mar-1994 12:35:19.00
Declination:	-3.1


**_Important Warning_**: The data are comma-separated, and for the numbers, the decimal separator is `.`. When parsing the data, do make sure the `Locale` is set correctly. For example, if the `Locale` is set to `fr_FR`, the decimal separator will be `,`... which can indeed lead to unexpected results. See in the code above how this is used, the number format named `NF`.

In [9]:
System.out.printf("User Locale: %s_%s\n", System.getProperty("user.language"), System.getProperty("user.country"));

User Locale: en_US


java.io.PrintStream@395716dc

Good luck!